# Matchup Analysis

### MSP 7.1.2017

It's time. Fivethirtyeight has their model, we need one of our own.

In [1]:
from IPython.display import HTML
HTML('''<script>code_show=true; function code_toggle() { if (code_show){ $('div.input').hide();} else { $('div.input').show(); } code_show = !code_show} $( document ).ready(code_toggle);</script><form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
# boilerplate imports

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

In [5]:
# imports for scraping

from bs4 import BeautifulSoup

import requests


In [6]:
# get twins data

r  = requests.get("http://www.fangraphs.com/teams/twins/schedule")

data = r.text

soup = BeautifulSoup(data)


table_div = soup.find('div' , {"class":"team-schedule-table"})
table = table_div.find('table')

# The first tr contains the field names.
headings = [th.get_text() for th in table.find("tr").find_all("th")]

datasets = []
for row in table.find_all("tr")[1:]:
    dataset = zip(headings, (td.get_text() for td in row.find_all("td")))
    datasets.append(dataset)



In [8]:
# check out the first game of the season
print datasets[0]

[(u'Date', u'Apr 3, 20174/3'), (u'', u'vs'), (u'Opp', u'KCR'), (u'MINWin Prob', u'50.9%'), (u'W/L', u'W'), (u'MINRuns', u'7'), (u'OppRuns', u'1'), (u'MIN Starter', u'Ervin Santana'), (u'Opp Starter', u'Danny Duffy')]


### Goal: make a model for teams facing each other that can be built from.
